In [2]:
import pandas as pd
import re
from     gensim.parsing.preprocessing     import     STOPWORDS,     strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, strip_short, stem_text
import pickle
import spacy #import en_core_web_sm
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chiso\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [30]:
zalando = pd.read_json(r"..\Zalando_Analytics_HWR_BIPM\twitterdata\2021-03-05_en_@zalando.json")
asos = pd.read_json(r"..\Zalando_Analytics_HWR_BIPM\twitterdata\2021-03-05_en_asos.json")
bohoo = pd.read_json(r"..\Zalando_Analytics_HWR_BIPM\twitterdata\2021-03-05_en_bohoo.json")

In [9]:
zalando.head()

,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified
0,1323291681899294721,133,GreenerRetail,Who and what has made our greener retailing tr...,0,Planet Earth,05-Mar-2021,70,False
1,1259761930304196610,1,fattybabycat,".@Zalando offers nothing more than scripted, c...",0,,05-Mar-2021,0,False
2,835467350249963521,2296,piotrkarwatka,We've got a great talk with Fabian Wesner on h...,0,"Wrocław, Polska",05-Mar-2021,1347,False
3,58485935,114,MissyDawn27586,@Zalando recieved wrong item... don’t know how...,0,,05-Mar-2021,27,False
4,1059339103928492033,11725,CarolCooney7,@DrewLawDesign @Zalando have sent me a few de...,0,,05-Mar-2021,457,False


In [10]:
asos.head()

,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified
0,91800594,97473,_phelma,Quickly came to return my ASOS package &amp; t...,0,"London, England",05-Mar-2021,2070,False
1,78974281,5481,ZiyandaO,@ASOS_HeretoHelp why has my account been block...,0,"London, England",05-Mar-2021,821,False
2,15790423,229382,ASOS,@_justollie @PostOffice @RoyalMailHelp @RoyalM...,0,London,05-Mar-2021,1085110,True
3,1208987533192957953,7184,RHBlackMan,Birthday Tingz. #LockdownStyle #InstagramSawIt...,0,Barbados,05-Mar-2021,323,False
4,1216824142726561792,28,SAM92075802,ASOS I’ve been awaiting for a parcel since 15 ...,0,,05-Mar-2021,15,False


In [11]:
bohoo.head()

,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified
0,136420228,152284,MissNellyNels,@fragileceour Wahala for who no be Bohoo chic,0,,05-Mar-2021,5744,False
1,136420228,152284,MissNellyNels,Bohoo take all my money https://t.co/3a7hbF5OX5,0,,05-Mar-2021,5744,False
2,1354040830592335874,800,thepurpsoul,@EB0YGGUKZ @swagijoon Twitter bohoo👎😤,0,Bangtan's Heart,05-Mar-2021,45,False
3,930457106,172758,renethelover,@neeuQcMxelA @enctrI you clearly view the issu...,0,,05-Mar-2021,2662,False
4,1174750574333632512,10807,driphaver,"@240pee bohoo cry about it,,\nWHAT A PUTT TO T...",0,Jo#8762 | id/thrasher,05-Mar-2021,633,False


In [12]:
print('zalando size:', zalando.shape)
print('asos size:', asos.shape)
print('bohoo size:', bohoo.shape)

zalando size: (32, 9)
asos size: (3000, 9)
bohoo size: (72, 9)


In [13]:
zalando.tweet[0]

'Who and what has made our greener retailing tracker this week? Step forward\xa0@EllasKitchenUK,\xa0@MarsUK, @NestleUKI,\xa0@Taylors,\xa0@suezUK, @tesconews,\xa0@mondigroup, @TanyaAshton7,\xa0@FietskoeriersNL,\xa0@Zalando,\xa0@benetton, and\xa0@ZARA\xa0https://t.co/KyPaEUVIG4 https://t.co/x32G8Md3SD'

In [14]:
zalando.tweet[30]

'„Tailoring rituals to your team“ How We (Re)Designed Our Community Rituals by Ekaterina Klykova @Zalando https://t.co/xVTZtyIDiH'

In [15]:
zalando.tweet[20]

'@SalmiSarppa @Zalando Any tips would be highly appreciated. Or do those pants include panties as well?'

In [16]:
asos.tweet[1]

'@ASOS_HeretoHelp why has my account been blocked and order cancelled? I also have another order due for delivery when will I receive that as I can’t track it anymore? What is that even about!? Disappointing as it’s only my 3rd time using your services. Please respond!'

In [21]:
asos.tweet[1000]

'I wish this was a viable option for clients 😂 https://t.co/kiWIcuwWYK'

In [23]:
asos.tweet[2989]

'@ASOS Because I’ll have gone almost another 6 during lockdown3.0 without @ItsSANK and I’ll need some new clothes for when we’re finally reunited ✨ #ASOSTreatMe'

In [25]:
bohoo.tweet[3]

'@neeuQcMxelA @enctrI you clearly view the issue "racism against whites in SA" as the bigger issue than the systemic racism against bipoc\'s all around the world. i\'m so sorry your life isn\'t included in the # black lives matter  bohoo be glad that u don\'t have to fight for your rights in 2021 !!'

In [28]:
bohoo.tweet[39]

'people who call others boring will always be a surprise for me. bohoo ur the most funnest. coolest. chillest. big deal bruh\nBut someone else’s idea of fun could be completely opposite to urs. maybe ur boring for them. how about that'

In [29]:
bohoo.tweet[70]

'@howlinmadmaddy @tfwPhoebe Bohoo literally ryu is shirtless and called "sexy ryu" and i dont see you bitching'